# Build solvated model of Villin N68H with OpenMM

## Prerequisites

Before running this notebook, ensure you have OpenMM installed:
```bash
conda create --platform osx-arm64 -n openmm-env python=3.13 conda-forge::openmm
conda activate openmm-env
```

You'll also need a PDB file named `villin-n68h.pdb` in your working directory. This should contain the initial coordinates of your molecular system.

## Complete Code

In [ ]:
# Import necessary modules
from openmm import app
from openmm import unit
from openmm import VerletIntegrator

# Load molecular structure
pdb = app.PDBFile("villin-n68h.pdb")

# Define force field parameters
forcefield = app.ForceField(
    "amber99sb.xml", "tip3p.xml"
)

# Create physical system with simulation parameters
modeller = app.Modeller(
    pdb.topology, pdb.positions
)

# Add hydrogens
modeller.addHydrogens(forcefield)

# Create box and solvate
modeller.addSolvent(
    forcefield,
    model='tip3p',
    padding=1*unit.nanometer
)

# Create physical system with simulation parameters
system = forcefield.createSystem(
    modeller.topology, nonbondedMethod=app.PME
)

# Set up Verlet integrator for NVE ensemble
integrator = VerletIntegrator(0.001*unit.picoseconds)
simulation = app.Simulation(
    modeller.topology, system, integrator
)
simulation.context.setPositions(modeller.positions)

# Minimize energy to remove bad contacts
simulation.minimizeEnergy(maxIterations=100)

# Save file
positions = simulation.context.getState(positions=True).getPositions()
app.PDBFile.writeFile(
    simulation.topology, positions, open("villin-n68h_solv.pdb", "w")
)